In [1]:
import pandas as pd
import numpy as np
import requests
import pyodbc
import arcpy
import os
from arcgis.features import FeatureLayer
import glob
import shutil


# Need to define datatypes so that FIPS code doesn't get cast as int and drop leading 0s
dtypes = {
    'YEAR' : str,
    'STATE': str,
    'GEOGRAPHY': str,
    'GEOID': str,
    'TRPAID':str,
    'NEIGHBORHOOD': str
}


In [2]:
#add a leading 0 to the TRPAID column if the length is 15
def add_leading_zero_based_on_condition(row):
    if row['sample_level'] == 'tract' and len(row['TRPAID']) == 14:
        return row['TRPAID'].zfill(15)
    elif row['sample_level'] == 'block group' and len(row['TRPAID']) == 15:
        return row['TRPAID'].zfill(16)
    else:
        return row['TRPAID']

    combined_csv['TRPAID'] = combined_csv['TRPAID'].astype(str)
    combined_csv['TRPAID'] = combined_csv.apply(add_leading_zero_based_on_condition, axis=1)
    combined_csv
    combined_csv['state'] = combined_csv['state'].astype(str).str.zfill(2)
    combined_csv['county'] = combined_csv['county'].astype(str).str.zfill(3)
    # Drop rows with a missing value
    combined_csv = combined_csv.dropna(subset=['value'])
    combined_csv.to_csv('combined_csv.csv', index=False)

In [3]:
combined_csv = pd.read_csv(r'C:\Users\amcclary\Documents\GitHub\Housing\Scripts\Summarized_Data\education.csv')
combined_csv['TRPAID'] = combined_csv['GEO_ID'].astype(str) + combined_csv['census_geom_year'].astype(str)

In [4]:
#add a leading 0 to the TRPAID column if the length is 15
def add_leading_zero_based_on_condition(row):
    if row['sample_level'] == 'tract' and len(row['TRPAID']) == 14:
        return row['TRPAID'].zfill(15)
    elif row['sample_level'] == 'block group' and len(row['TRPAID']) == 15:
        return row['TRPAID'].zfill(16)
    else:
        return row['TRPAID']
combined_csv['TRPAID'] = combined_csv.apply(add_leading_zero_based_on_condition, axis=1)

In [5]:
clean_csv = combined_csv.copy()

# Convert all columns of type 'object' to 'string'
clean_csv = clean_csv.astype({col: 'string' for col in clean_csv.select_dtypes(include='object').columns})

clean_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155 entries, 0 to 154
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   GEO_ID             155 non-null    int64  
 1   value              155 non-null    float64
 2   MarginOfError      0 non-null      float64
 3   state              155 non-null    int64  
 4   county             155 non-null    int64  
 5   tract              155 non-null    int64  
 6   block_group        0 non-null      float64
 7   variable_code      155 non-null    string 
 8   variable_name      155 non-null    string 
 9   variable_category  155 non-null    string 
 10  year_sample        155 non-null    int64  
 11  sample_level       155 non-null    string 
 12  dataset            155 non-null    string 
 13  census_geom_year   155 non-null    float64
 14  TRPAID             155 non-null    string 
 15  NEIGHBORHOOD       155 non-null    string 
 16  Data_Scale         0 non-n

In [ ]:
clean_csv = clean_csv.loc[:, ~clean_csv.columns.str.contains('^Unnamed')]
clean_csv['NEIGHBORHOOD'] = clean_csv['NEIGHBORHOOD'].fillna("")

In [6]:

clean_csv['GEO_ID'] = clean_csv['GEO_ID'].astype(str)
type_mapping = {
    'int64': 'LONG',
    'float64': 'DOUBLE',
    'object': 'TEXT',
    'string': 'TEXT',
    'datetime64[ns]': 'DATE'
}

# Set up geodatabase and output table name
gdb_path = r"F:\GIS\PROJECTS\ResearchAnalysis\Demographics\Workspace.gdb"
output_table = "Housing_Metrics_Education"
output_path = f"{gdb_path}\\{output_table}"
if arcpy.Exists(output_path):
    arcpy.management.Delete(output_path)
    print(f"Deleted existing table: {output_table}")
# Create the table in the geodatabase
arcpy.management.CreateTable(gdb_path, output_table)

# Add fields based on DataFrame dtypes
for col_name, dtype in clean_csv.dtypes.items():
    arcgis_type = type_mapping.get(str(dtype), 'TEXT')  # Default to TEXT if dtype is unknown
    if arcgis_type == 'TEXT':
        arcpy.management.AddField(output_path, col_name, arcgis_type, field_length=255)
    else:
        arcpy.management.AddField(output_path, col_name, arcgis_type)

# Insert data into the table
with arcpy.da.InsertCursor(output_path, clean_csv.columns.tolist()) as cursor:
    for _, row in clean_csv.iterrows():
        cursor.insertRow(row.tolist())

print(f"Table '{output_table}' created and populated in {gdb_path}")

Table 'Housing_Metrics_Education' created and populated in F:\GIS\PROJECTS\ResearchAnalysis\Demographics\Workspace.gdb


In [9]:
import pandas as pd

# Example DataFrame
df = pd.DataFrame({
    'A': [5, 2, 2, 5],
    'B': [5, 6, 6, 8],
    'C': ['x', 'y', 'y', 'z']
})

# Remove duplicate rows
df_unique = df.drop_duplicates()

print(df_unique)


   A  B  C
0  5  5  x
1  2  6  y
3  5  8  z


In [ ]:
#remove duplicate rows from clean_csv
clean_csv = clean_csv.drop_duplicates()
unique_values = clean_csv